<a href="https://colab.research.google.com/github/fatimahaidara/NLP_labs/blob/main/Fatoumata_Intro_to_wordvectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 2: Introduction to wordvectors </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [6]:
import io, sys
import numpy as np

In [7]:
def load_vectors(filename):
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.asarray([float(x) for x in tokens[1:]])
    return data

In [8]:
# from google.colab import drive
# drive.mount('/content/drive')

In [9]:
# path = "/content/drive/MyDrive/NLP_Week_1_Labs_2022/session2/session2/"

In [10]:
# Loading word vectors

print('')
print(' ** Word vectors ** ')
print('')

'''
word_vectors is a dictionary that maps words to their numerical word vector
[word (string)] = [np-array] 
'''
word_vectors = load_vectors('wiki.en.vec')

tree_vector = word_vectors['tree']
print(type(tree_vector), len(tree_vector))


 ** Word vectors ** 

<class 'numpy.ndarray'> 300


In [11]:
## This function computes the cosine similarity between vectors u and v

def cosine(u, v):
    '''
    Parameters:
    u : 1-D numpy array
    v : 1-D numpy array 
    
    Returns:
    cos (float) : value of the cosine similairy between vectors u, v 
    '''

    cos = (np.dot(u,v))/(np.dot(np.linalg.norm(u),np.linalg.norm(v)))
    
    return cos 


In [12]:
# compute similarity between words

print('similarity(apple, apples) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['apples']))
print('similarity(apple, banana) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['banana']))
print('similarity(apple, tiger) = %.3f' %
      cosine(word_vectors['apple'], word_vectors['tiger']))

similarity(apple, apples) = 0.637
similarity(apple, banana) = 0.431
similarity(apple, tiger) = 0.212


In [13]:
## Functions for nearest neighbor
## This function returns the word corresponding to 
## nearest neighbor vector of x
## The list exclude_words can be used to exclude some
## words from the nearest neighbors search

def nearest_neighbor(x, word_vectors, exclude_words=[]):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    word_vectors (Python dict): {word (string): np-array of word vector}
    exclude_words (list of strings): words to be excluded from the search
    
    Returns:
    best_word (string) : the word whose word vector is the nearest neighbour 
    to the word vector of x
    '''
    best_score = -1.0
    best_word = None

    for word,vec in word_vectors.items():
      if not word in exclude_words and best_score != 0.99:
        if cosine(word_vectors[word],x) > best_score:
          best_score = cosine(word_vectors[word],x)
          best_word = word
            
    return best_word

In [14]:
nearest_neighbor(word_vectors['why'],word_vectors,exclude_words =['why'])

'really'

In [15]:
print('')
print('The nearest neighbor of cat is: ' ,
      nearest_neighbor(word_vectors['cat'], word_vectors, exclude_words = ['cat', 'cats']))


The nearest neighbor of cat is:  dog


#### Hint (using python priorty queues with the heapq datastructure): 
if you don't want to store all the words and scores you can use the priortiy queue and only store the best K element so far. 

In [16]:
import heapq
## This function return the words corresponding to the
## K nearest neighbors of vector x.
## You can use the functions heappush and heappop.

def knn(x, word_vectors, k):
    '''
    Parameters:
    x (string): word to find its nearest neighbour 
    word_vectors (Python dict): {word (string): np-array of word vector}
    k (int): number of nearest neighbours to be found
    
    Returns: 
    k_nearest_neighbors (list of tuples): [(score, word), (score, word), ....]
    '''
    exclude_words = [x]
    # heapq.heapify(li)
    k_nearest_neighbors = []
    for i in range(k):
      word = nearest_neighbor(word_vectors[x], word_vectors, exclude_words)

      best_score = cosine(word_vectors[word],word_vectors[x])

      k_nearest_neighbors.append((best_score,word)) 
      exclude_words.append(word)
    
    return k_nearest_neighbors

In [17]:
knn_cat = knn('cat', word_vectors, 5)
print('')
print('cat')
print('--------------')
for score, word in knn('cat', word_vectors, 5):
    print (word + '\t%.3f' % score)


cat
--------------
cats	0.732
dog	0.638
pet	0.573
rabbit	0.549
dogs	0.538


#### Hint: 
To find the analogies, we find the nearest neighbour associated with the wordvector d
$$ d = \frac{c}{\Vert {c} \Vert} + \frac{b}{\Vert {b} \Vert} - \frac{a}{\Vert {a} \Vert}$$


In [18]:
## This function return the words d, such that a:b and c:d
## verifies the same relation

def analogy(a, b, c, word_vectors):
    '''
    Parameters:
    a (string): word a
    b (string): word b
    c (string): word c
    word_vectors (Python dict): {word (string): np-array of word vector}
    
    Returnrs: 
    the word d (string) associated with c such that c:d is similar to a:b 
    
    '''
        
    # convert words to lower case
    a, b, c = a.lower(),b.lower(), c.lower()
    
    # Get the word embeddings v_a, v_b and v_c (≈1-3 lines)
    e_a, e_b, e_c = word_vectors[a], word_vectors[b], word_vectors[c]
    
    words = word_vectors.keys()
    max_cosine_sim = -100              # Initialize max_cosine_sim to a large negative number
    best_d = None                   # Initialize best_word with None, it will help keep track of the word to output

    for w in words:        
        if w in [a, b, c] :
            continue
        
        similarity = cosine((e_b - e_a), (word_vectors[w] - e_c))
        
        # If the cosine_sim is more than the max_cosine_sim seen so far,
        if similarity > max_cosine_sim:
            max_cosine_sim = similarity
            best_d = w
        
    return best_d
    
    # d = (word_vectors[c]/np.linalg.norm(word_vectors[c])) + (word_vectors[b]/np.linalg.norm(word_vectors[b])) - (word_vectors[a]/np.linalg.norm(word_vectors[a]))
    # word = [k for k, v in word_vectors.items() if cosine(word_vectors[a], word_vectors[b]) == cosine(word_vectors[c], word_vectors[k])]
    # # np.array_equal(v,d)

    # return word

In [19]:
# Word analogies
print('')
print('france - paris + rome = ' , analogy('paris', 'france', 'rome', word_vectors))


france - paris + rome =  italy


## A word about biases in word vectors

In [20]:
## A word about biases in word vectors:

print('')
print('similarity(genius, man) = %.3f' %
      cosine(word_vectors['man'], word_vectors['genius']))
print('similarity(genius, woman) = %.3f' %
      cosine(word_vectors['woman'], word_vectors['genius']))


similarity(genius, man) = 0.445
similarity(genius, woman) = 0.325


In [21]:
## Compute the association strength between:
##   - a word w
##   - two sets of attributes A and B

def association_strength(w, A, B, word_vectors):
    '''

    Parameters:
    w (string): word w
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    
    Returnrs: 
    strength (float): the value of the association strength 

    '''
    
    strength = 0.0
    part_a = 0.0
    part_b = 0.0 
    
    card_A = len(A)
    card_B = len(B)
    for i in A:
      part_a += (cosine(word_vectors[i], word_vectors[w]))/card_A 
    for j in B:
      part_b += (cosine(word_vectors[j], word_vectors[w]))/card_B 
    
    strength = part_a - part_b
    return strength            
                       

In [22]:
association_strength('ok',['true','false'],['no','yes'],word_vectors)

-0.12385480567297091

In [23]:
## Perform the word embedding association test between:
##   - two sets of words X and Y
##   - two sets of attributes A and B

def weat(X, Y, A, B, word_vectors):
    '''
    Parameters:
    X (list of strings): The words belonging to set X
    Y (list of strings): The words belonging to set Y
    A (list of strings): The words belonging to set A
    B (list of strings): The words belonging to set B
    vectors (Python dict): {word (string): np-array of word vector}
    
    Returns: 
    score (float): the value of the group association strength  
    '''
    strength_X = 0.0
    strength_Y = 0.0 
    score = 0.0
    for i in X:
      strength_X += association_strength(i,A,B,word_vectors)
    for j in Y:
      strength_Y += association_strength(j,A,B,word_vectors)

    score = strength_X - strength_Y

    return score

In [24]:
## Replicate one of the experiments from:
##
## Semantics derived automatically from language corpora contain human-like biases
## Caliskan, Bryson, Narayanan (2017)

career = ['executive', 'management', 'professional', 'corporation', 
          'salary', 'office', 'business', 'career']
family = ['home', 'parents', 'children', 'family',
          'cousins', 'marriage', 'wedding', 'relatives']
male = ['john', 'paul', 'mike', 'kevin', 'steve', 'greg', 'jeff', 'bill']
female = ['amy', 'joan', 'lisa', 'sarah', 'diana', 'kate', 'ann', 'donna']

print('')
print('Word embedding association test: %.3f' %
      weat(career, family, male, female, word_vectors))


Word embedding association test: 0.847


## Word translation using word vectors

In the following, we will use word vectors in English and French to translate words from English to French. The idea is to learn a linear function that maps English word vectors to their correponding French word vectors. To learn this linear mapping, we will use a small bilingual lexicon, that contains pairs of words in English and French that are translations of each other.

The following function will load the small English-French bilingual lexicon:

In [25]:
def load_lexicon(filename):
    '''
    Parameters:
    filename(string): the path of the lexicon
    
    Returns:
    data(list of pairs of string): the bilingual lexicon
    '''
    fin = io.open(filename, 'r', encoding='utf-8', newline='\n')
    data = []
    for line in fin:
        a, b = line.rstrip().split(' ')
        data.append((a, b))
    return data

In [26]:
word_vectors_en = load_vectors('wiki.en.vec')
word_vectors_fr = load_vectors('wiki.fr.vec')
lexicon = load_lexicon("lexicon-en-fr.txt")
print(lexicon[:5])

[('the', 'le'), ('the', 'les'), ('the', 'la'), ('and', 'et'), ('was', 'fut')]


In [27]:
len(word_vectors_en['the'])

300

In [28]:
# We split the lexicon into a train and validation set
train = lexicon[:5000]
valid = lexicon[5000:5100]

The following function will learn the mapping from English to French. The idea is to build two matrices $X_{\text{en}}$ and $X_{\text{fr}}$, and to find a mapping $M$ that minimizes $||X_{\text{en}} W - X_{\text{fr}} ||_2$. In numpy, this mapping can be obtained by using the `numpy.linalg.lstsq` function.

In [29]:
def align(word_vectors_en, word_vectors_fr, lexicon):
    '''
    Parameters:
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    lexicon(list of pairs of string): bilingual training lexicon
    
    Returns
    mapping(np.array): the mapping from English to French vectors
    '''
    x_en, x_fr = [], []
    for (en,fr) in lexicon:
      # (en,fr) = i
      vec_en = word_vectors_en[en]
      vec_fr = word_vectors_fr[fr]
      x_en.append(vec_en)
      x_fr.append(vec_fr)

    mapping = np.linalg.lstsq(x_en,x_fr,rcond=None) 
    
    return mapping[0]

In [30]:
mapping = align(word_vectors_en, word_vectors_fr, lexicon)
mapping.shape

(300, 300)

Given a mapping, a set of word English word vector and French word vectors, the next function will translate the English word to French. To do so, we apply the mapping on the English word, and retrieve the nearest neighbor of the obtained vector in the set of French word vectors. The translation is then the corresponding French word.

In [40]:
def translate(word, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    word(string): an English word
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_fr(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    A string containing the translation of the English word
    '''
    word_en = word_vectors_en[word]
    vec = word_en@mapping
    word = nearest_neighbor(vec, word_vectors_fr)
    
    return word

In [41]:
print(translate("world", word_vectors_en, word_vectors_fr, mapping))
print(translate("machine", word_vectors_en, word_vectors_fr, mapping))
print(translate("learning", word_vectors_en, word_vectors_fr, mapping))

monde
machine
apprentissage


Finally, let's implement a function to evaluate this method on the validation lexicon:

In [42]:
def evaluate(valid, word_vectors_en, word_vectors_fr, mapping):
    '''
    Parameters:
    valid(a list of pairs of string): the validation lexicon
    word_vectors_en(dict: string -> np.array): English word vectors
    word_vectors_en(dict: string -> np.array): French word vectors
    mapping(np.array): the mapping from English to French vectors
    
    Returns
    Accuracy(float): the accuracy on the validation lexicon
    '''
    acc, n = 0.0, 0
    for en,fr in valid:
      french_word = translate(en, word_vectors_en, word_vectors_fr, mapping)
      if fr == french_word:
        n += 1
    acc= n/len(valid)
    return acc

In [43]:
evaluate(valid, word_vectors_en, word_vectors_fr, mapping)

0.64